In [499]:
import pandas as pd

In [589]:
interest_rate = pd.read_csv("./대출금리.csv" )
buy_index = pd.read_csv("./아파트매매가격지수.csv")
year_index = pd.read_csv("./아파트전세가격지수.csv")
seoul_gu = pd.read_csv("./서울시 구.csv")
buy_year_rate = pd.read_csv("./아파트 매매가격대비 전세가격비율.csv")

# 1. 전처리

### 1. 주택금리 전처리

##### 1.1) 주택담보대출 (연리%)만 가져오기

In [590]:
interest_rate2 = interest_rate[interest_rate['계정항목별'] == '주택담보대출 (연리%)'].transpose().reset_index(drop=False).iloc[1:]
interest_rate2.columns = ['날짜', '주택담보대출 (연리%)']

1.2) 날짜 형식 통일

In [591]:
interest_rate2['날짜'] = pd.to_datetime(interest_rate2['날짜'])

In [592]:
interest_rate3 = interest_rate2.set_index('날짜')
interest_rate3 = interest_rate3.applymap(lambda x: pd.to_numeric(x, errors='coerce')) #수치형 변환 추가

### 2. 아파트 매매가격지수 전처리

#### 2.1) 서울시만 가져오기

In [593]:
gu_list = seoul_gu['구'].to_list()

In [594]:
buy_index2 = buy_index.rename(columns={'구분':'날짜'}, inplace = 'TRUE')
buy_index3 = buy_index.set_index('날짜')[gu_list].iloc[1:]

#### 2.1) 월별 평균 지수로 변경

In [595]:
buy_index3 = buy_index3.reset_index()
buy_index3['날짜'] = pd.to_datetime(buy_index3['날짜'])
buy_index3['month'] = buy_index3['날짜'].dt.month
buy_index3['year'] = buy_index3['날짜'].dt.year

In [596]:
# 수치형으로 변경
forint = buy_index3.columns.difference(['year', 'month','날짜'])
buy_index3[forint] = buy_index3[forint].apply(pd.to_numeric, errors='coerce')

In [597]:
#월별 평균지수
buy_index3 = buy_index3.groupby(['year','month']).mean().reset_index()
buy_index3['날짜'] = buy_index3['year'].astype(str) + '-' + buy_index3['month'].astype(str)
buy_index3['날짜'] = pd.to_datetime(buy_index3['날짜'], format='%Y-%m')
buy_index3 = buy_index3.drop(columns = ['month','year'], axis = 1)

<ipython-input-597-d550126ba564>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  buy_index3 = buy_index3.groupby(['year','month']).mean().reset_index()


In [598]:
#2015년 1월1일 ~ 2023년 11월 1일까지만 가져오기
buy_index4 = buy_index3[(buy_index3['날짜'] >= '2015-01-01') & (buy_index3['날짜'] <= '2023-11-01')]
buy_index = buy_index4.set_index('날짜')

### 3. 아파트 전세가격지수 전처리

In [599]:
year_index2 = year_index.rename(columns={'구분':'날짜'}, inplace = 'TRUE')
year_index3 = year_index.set_index('날짜')[gu_list].iloc[1:]
year_index3 = year_index3.reset_index()
year_index3['날짜'] = pd.to_datetime(year_index3['날짜'])
year_index3['month'] = year_index3['날짜'].dt.month
year_index3['year'] = year_index3['날짜'].dt.year

In [600]:
# 수치형으로 변경
forint = year_index3.columns.difference(['year', 'month','날짜'])
year_index3[forint] = year_index3[forint].apply(pd.to_numeric, errors='coerce')

#월별 평균지수
year_index3 = year_index3.groupby(['year','month']).mean().reset_index()
year_index3['날짜'] = year_index3['year'].astype(str) + '-' + year_index3['month'].astype(str)
year_index3['날짜'] = pd.to_datetime(year_index3['날짜'], format='%Y-%m')
year_index3 = year_index3.drop(columns = ['month','year'], axis = 1)

<ipython-input-600-77256221b3b7>:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  year_index3 = year_index3.groupby(['year','month']).mean().reset_index()


In [601]:
#2015년 1월1일 ~ 2023년 11월 1일까지만 가져오기
year_index4 = year_index3[(year_index3['날짜'] >= '2015-01-01') & (year_index3['날짜'] <= '2023-11-01')]

### 4. 전세/매매가격 지수 비율 전처리

In [602]:
buy_year_rate2 = buy_year_rate.set_index('지역명').transpose()[gu_list].reset_index().rename(columns = {'index' : "날짜"})
buy_year_rate2 = buy_year_rate2.rename(columns = {'index' : "날짜"})
buy_year_rate2['날짜'] = pd.to_datetime(buy_year_rate2['날짜'])
buy_year_rate2.columns.name = None

In [603]:
#2015년 1월1일 ~ 2023년 11월 1일까지만 가져오기
buy_year_rate3 = buy_year_rate2[(buy_year_rate2['날짜'] >= '2015-01-01') & (buy_year_rate2['날짜'] <= '2023-11-01')]

### 5. 임대료 데이터 만들기
(전세 지수 : year_index)* (전세/매매가격 지수 : year_buy_rate ) * (주택담보대출금리(%) : interest_rate / 1200 + 1)

In [604]:
year_buy_rate4 = buy_year_rate3.set_index('날짜') #전세/매매가격
year_buy_rate4 = year_buy_rate4.applymap(lambda x: pd.to_numeric(x, errors='coerce'))

In [605]:
#1) (전세/매매가격 지수)/100 * (주택담보대출금리(%) / 1200 + 1)
columns = year_buy_rate4.columns
year_buy_rate4 = year_buy_rate4.reset_index()
for i in columns  :
  year_buy_rate4[i] = year_buy_rate4[i]/100 * (interest_rate3.reset_index()['주택담보대출 (연리%)'] /1200+ 1)

In [606]:
#2) (전세 지수) * 위 블록 결과(year_buy_rate4)
year_index = year_index4.set_index('날짜') ##전세 지수
year_buy_rate4 = year_buy_rate4.set_index('날짜') # (전세/매매가격 지수) * (주택담보대출금리(%) / 12 /100+1)

rent = year_index * year_buy_rate4

In [607]:
#월별 임대료
rent

,용산구,서초구,강남구,송파구,강동구,광진구,중랑구,노원구,도봉구,강북구,...,서대문구,은평구,마포구,강서구,양천구,영등포구,구로구,금천구,관악구,동작구
날짜,,,,,,,,,,,,,,,,,,,,,
2015-01-01,39.482507,43.932983,40.739963,40.622497,48.670130,48.725391,45.307146,44.420365,47.243908,45.601205,...,54.003349,45.841257,43.405651,46.844967,42.297071,42.865049,46.032226,47.716846,48.499949,47.015387
2015-02-01,40.016060,45.605494,41.326983,41.482420,50.577602,49.364530,46.033522,44.725482,47.737787,46.235298,...,54.846813,46.482276,43.905654,48.016591,42.822289,43.244626,46.375936,47.974610,49.189617,47.954254
2015-03-01,40.611098,46.797721,42.562276,42.340950,52.459130,50.426876,47.162405,45.590722,48.680088,47.229313,...,55.672727,47.515932,44.709619,49.694138,43.328105,44.098639,47.401061,48.289750,49.948310,49.091841
2015-04-01,41.105622,48.008414,43.346127,43.004236,54.561999,50.783251,48.557900,46.394881,49.924153,47.791295,...,56.894894,48.429144,45.814088,51.014058,43.414135,45.311274,48.553271,49.148974,50.939945,50.246923
2015-05-01,41.312663,48.515935,44.075316,43.678770,55.432245,51.229898,49.514486,47.304375,50.927334,48.560986,...,57.595490,49.064830,46.725262,52.561195,43.919114,46.387613,49.315868,49.658572,51.540988,51.299667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-01,35.618963,37.865041,33.479338,37.029499,43.739673,42.768088,52.042882,41.999844,45.691384,48.566801,...,49.224926,48.209225,44.591827,44.559009,41.614912,45.169040,45.743355,50.287922,46.223445,42.656759
2023-08-01,35.442727,37.851948,33.772304,37.411975,44.483132,43.225946,51.931769,41.977211,44.713363,48.570043,...,49.105738,48.252708,44.861488,44.976190,41.513154,45.110167,45.656519,50.206078,46.634820,43.396326
2023-09-01,35.684990,38.212922,33.923636,37.799434,44.938198,43.773764,51.870918,42.035876,44.458597,49.007629,...,49.233838,48.103639,45.274565,45.340461,41.683673,45.433341,46.170561,50.546086,47.407060,44.121155


In [608]:
#월별 매매가격지수
buy_index

,용산구,서초구,강남구,송파구,강동구,광진구,중랑구,노원구,도봉구,강북구,...,서대문구,은평구,마포구,강서구,양천구,영등포구,구로구,금천구,관악구,동작구
날짜,,,,,,,,,,,,,,,,,,,,,
2015-01-01,58.900000,59.075,56.550000,55.000000,58.625000,58.800,62.500,51.000000,56.000000,58.400,...,60.225,59.40,53.675000,55.100,53.800000,51.900000,53.725,60.175000,59.500,53.900000
2015-02-01,58.933333,59.200,56.666667,55.033333,58.866667,58.800,62.500,51.133333,56.066667,58.500,...,60.400,59.50,53.733333,55.200,53.866667,51.966667,53.800,60.266667,59.500,53.933333
2015-03-01,59.100000,59.580,57.040000,55.180000,59.340000,59.080,62.640,51.400000,56.320000,58.640,...,60.780,59.78,54.020000,55.620,54.100000,52.040000,54.020,60.540000,59.700,54.220000
2015-04-01,59.350000,60.125,57.350000,55.425000,59.850000,59.275,62.900,51.775000,56.575000,58.925,...,61.150,60.10,54.375000,56.050,54.525000,52.150000,54.425,60.875000,60.050,54.625000
2015-05-01,59.475000,60.350,57.650000,55.575000,60.125000,59.375,63.125,51.900000,56.700000,59.050,...,61.325,60.30,54.550000,56.375,54.700000,52.450000,54.725,61.000000,60.325,54.825000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-01,98.220000,95.340,95.620000,91.440000,89.360000,93.440,91.480,85.220000,84.300000,88.800,...,88.300,91.88,92.200000,89.580,93.120000,93.020000,88.120,91.980000,90.100,90.180000
2023-08-01,98.125000,95.725,96.100000,92.200000,89.725000,93.425,91.075,84.875000,83.825000,88.700,...,88.225,91.25,92.550000,89.425,93.525000,93.100000,87.825,91.900000,89.600,90.225000
2023-09-01,98.425000,96.000,96.650000,93.050000,90.075000,93.675,90.800,84.700000,83.675000,88.750,...,88.350,91.10,92.950000,89.575,94.100000,93.450000,87.800,91.800000,89.575,90.475000


In [588]:
#월별 전세가격지수
year_index

,용산구,서초구,강남구,송파구,강동구,광진구,중랑구,노원구,도봉구,강북구,...,서대문구,은평구,마포구,강서구,양천구,영등포구,구로구,금천구,관악구,동작구
날짜,,,,,,,,,,,,,,,,,,,,,
2015-01-01,68.750000,70.600000,69.425,65.250000,77.925000,69.025000,68.275000,66.125,69.850000,66.800000,...,74.300000,68.000000,63.200000,67.825000,67.250,67.125,65.700,71.325000,68.600000,66.525
2015-02-01,69.033333,71.366667,69.700,65.733333,78.533333,69.366667,68.566667,66.300,70.133333,67.133333,...,74.833333,68.366667,63.533333,68.366667,67.500,67.500,65.900,71.533333,68.966667,66.800
2015-03-01,69.620000,72.280000,70.360,66.300000,79.740000,69.820000,69.200000,67.020,70.980000,67.840000,...,75.580000,69.080000,64.140000,69.460000,67.960,68.060,66.480,71.980000,69.540000,67.380
2015-04-01,70.125000,73.325000,71.175,66.950000,81.200000,70.125000,70.075000,67.825,72.025000,68.325000,...,76.375000,69.950000,65.050000,70.575000,68.350,68.875,67.400,72.525000,70.450000,68.300
2015-05-01,70.400000,73.625000,71.625,67.500000,81.875000,70.300000,70.700000,68.350,72.475000,68.800000,...,76.875000,70.350000,65.625000,71.175000,68.775,69.500,67.950,72.925000,71.000000,68.825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-01,84.580000,83.580000,81.140,81.740000,85.100000,84.040000,88.820000,85.540,85.820000,85.160000,...,88.100000,85.020000,85.500000,85.760000,85.380,89.200,82.560,87.340000,82.820000,82.760
2023-08-01,84.400000,83.725000,81.475,82.375000,86.050000,84.575000,88.475000,85.225,84.700000,85.175000,...,87.925000,84.775000,85.900000,86.050000,85.450,89.150,82.350,87.275000,83.100000,83.500
2023-09-01,84.775000,84.225000,81.825,83.200000,86.675000,85.175000,88.300000,85.200,84.450000,85.550000,...,88.100000,84.525000,86.500000,86.525000,85.875,89.575,82.850,87.500000,83.675000,84.325
